In [1]:
import sys
from pyspark.sql import functions as f

In [2]:
print(sys.version)

3.7.16 (default, Jan 17 2023, 16:06:28) [MSC v.1916 64 bit (AMD64)]


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .appName("MySparkApplication") \
    .config("spark.driver.memory", "3g") \
    .config("spark.executor.memory", "3g") \
    .getOrCreate()


In [5]:


parquet_output_path = "Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_review.parquet" 


parquet_output_path = "Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_business.parquet"
df_business.write.mode("overwrite").format("parquet").save(parquet_output_path)

parquet_output_path = "Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_checkin.parquet"
df_checkin.write.mode("overwrite").format("parquet").save(parquet_output_path)

parquet_output_path = "Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_review.parquet"
df_review.write.mode("overwrite").format("parquet").save(parquet_output_path)

parquet_output_path = "Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_tip.parquet"
df_tip.write.mode("overwrite").format("parquet").save(parquet_output_path)

parquet_output_path = "Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_user.parquet"
df_user.write.mode("overwrite").format("parquet").save(parquet_output_path)

In [6]:
df_user = spark.read.option("inferSchema","true").parquet("Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_user.parquet")
df_user_part = df_user.repartition(40)

df_business = spark.read.parquet("Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_business.parquet")

df_checkin = spark.read.parquet ("Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_checkin.parquet" )
df_review = spark.read.parquet  ("Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_review.parquet"  )
df_tip = spark.read.parquet     ("Downloads/data_2/data_2/json_to_parquet/yelp_academic_dataset_tip.parquet"     )
  

In [7]:
display(df_user)

DataFrame[average_stars: double, compliment_cool: bigint, compliment_cute: bigint, compliment_funny: bigint, compliment_hot: bigint, compliment_list: bigint, compliment_more: bigint, compliment_note: bigint, compliment_photos: bigint, compliment_plain: bigint, compliment_profile: bigint, compliment_writer: bigint, cool: bigint, elite: string, fans: bigint, friends: string, funny: bigint, name: string, review_count: bigint, useful: bigint, user_id: string, yelping_since: string]

In [8]:
df_user_part.createOrReplaceTempView("user")

<h4>**Find the top 3 users based on their total number of reviews:**</h5>

In [9]:
spark.sql("select user_id,name,review_count from user order by review_count desc limit 3").show()

+--------------------+------+------------+
|             user_id|  name|review_count|
+--------------------+------+------------+
|Hi10sGSZNxQH3NLyW...|   Fox|       17473|
|8k3aO-mPeyhbR5HUu...|Victor|       16978|
|hWDybu_KvYLSdEFzG...| Bruce|       16567|
+--------------------+------+------------+



In [10]:
df = df_business
!pip install pandas
import pandas as pd


<h4>**Find the top 10 users with the most fans:

In [11]:
spark.sql("Select user_id,name,fans from user order by fans desc limit 10").show()

+--------------------+-------+-----+
|             user_id|   name| fans|
+--------------------+-------+-----+
|37cpUoM8hlkSQfReI...|   Mike|12497|
|hizGc5W1tBHPghM5Y...|  Katie| 3642|
|Hi10sGSZNxQH3NLyW...|    Fox| 3493|
|JjXuiru1_ONzDkYVr...|Richard| 3243|
|j14WgRoU_-2ZE1aw1...| Daniel| 3138|
|VHdY6oG2JPVNjihWh...|Jessica| 2627|
|iLjMdZi0Tm7DQxX1C...|  Ruggy| 2547|
|lt7bNHl-TXziny4FE...|  Megan| 2451|
|fgwI3rYHOv1ipfVfC...|    Emi| 2424|
|ITa3vh5ERI90G_WP4...|  Peter| 2388|
+--------------------+-------+-----+



In [12]:
df_business.select("categories",).show(truncate=False)

+-----------------------------------------------------------------------------------------------------+
|categories                                                                                           |
+-----------------------------------------------------------------------------------------------------+
|Home & Garden, Home Decor, Furniture Stores, Shopping                                                |
|Cheesesteaks, Bubble Tea, Food, Burgers, Chicken Wings, Coffee & Tea, Restaurants                    |
|Restaurants, Chinese                                                                                 |
|Home Services, Real Estate Services, Real Estate                                                     |
|Shopping, Accessories, Women's Clothing, Fashion                                                     |
|Flowers & Gifts, Shopping, Florists                                                                  |
|Coffee & Tea, Breakfast & Brunch, Food, Restaurants            

In [13]:

pd = df_business.toPandas()

<h4>**Analyse top 10 categories by number of reviews:
    

In [14]:


df_business_cat = df_business.groupBy("categories").agg(f.count("review_count").alias("total_revenue_count"))
                                                    
df_business_cat.show()


df_business_cat.createOrReplaceTempView("cat")

from pyspark.sql.window import Window
spark.sql("select *, row_number() over (order by total_revenue_count desc) as rn from cat").show()

df_cat = df_business_cat.withColumn("rn",f.row_number().over(Window.orderBy(f.col('total_revenue_count').desc()))
                        )
x = df_cat.filter(df_cat['rn'] <= 10)
y = df_cat.filter(f.col('rn') <= 10)

x.show()


df_business_reviews = df_business.groupBy("categories").agg(f.count("review_count").alias("total_revenue_count")).orderBy(f.desc("total_revenue_count"))


+--------------------+-------------------+
|          categories|total_revenue_count|
+--------------------+-------------------+
|Automotive, Car D...|                  9|
|Screen Printing, ...|                  1|
|Parks, Active Lif...|                 12|
|Property Manageme...|                  1|
|Gold Buyers, Shop...|                  1|
|Restaurants, Chic...|                 12|
|Event Planning & ...|                 10|
|Hot Dogs, America...|                  1|
|Furniture Stores,...|                  5|
|Beauty & Spas, Sk...|                  1|
|Home Services, Ho...|                  1|
|Chicken Wings, Fa...|                  9|
|Restaurants, Amer...|                  1|
|Eyelash Service, ...|                  1|
|Venues & Event Sp...|                 13|
|Art Galleries, Je...|                  1|
|Veterinarians, Pe...|                  1|
|Tex-Mex, Mexican,...|                  1|
|Makeup Artists, H...|                  7|
|Coffee & Tea, Foo...|                  2|
+----------

<h4>**Analyse top business which have over 1000 reviews:

In [15]:
df_top = df_business_reviews.filter(f.col("total_revenue_count") >= 1000).orderBy(f.desc("total_revenue_count"))


In [16]:
df_top.show()

+--------------------+-------------------+
|          categories|total_revenue_count|
+--------------------+-------------------+
|Beauty & Spas, Na...|               1012|
+--------------------+-------------------+



In [17]:
pd.head(5)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,120 W Main St,"(None, None, None, None, None, None, None, Non...",xKoz9eM8hUELf5qixexeJQ,"Home & Garden, Home Decor, Furniture Stores, S...",Plainfield,"(10:0-18:0, None, 10:0-18:0, 12:0-17:0, None, ...",1,39.703082,-86.402772,African Plum Home & More,46168,5,4.5,IN
1,2224 E Fletcher Ave,"(None, None, u'none', {'touristy': False, 'hip...",yiE_Iym7A9wa6tyJHkWzhw,"Cheesesteaks, Bubble Tea, Food, Burgers, Chick...",Tampa,"(11:0-23:0, None, 12:0-23:0, 12:0-22:0, 11:0-2...",1,28.069623,-82.433078,Craft Fry Wing,33612,43,4.5,FL
2,7811 US 31 S,"(None, None, u'none', None, None, None, None, ...",7d9X9wnm_35UcdJRJNu_Sw,"Restaurants, Chinese",Indianapolis,"(10:30-23:0, 10:30-22:0, 10:30-23:0, 11:30-22:...",1,39.652946,-86.136276,China King,46227,22,4.0,IN
3,924 Arch St,"(None, None, None, None, None, None, None, Non...",CFPWVAFSP3Ktfe-cEXeKoA,"Home Services, Real Estate Services, Real Estate",Philadelphia,None,1,39.953134,-75.155747,Liberty Real Estate Associates,19107,15,1.5,PA
4,4317 Harding Pike,"(None, None, None, None, None, None, None, Fal...",EgZH_qXEQH_WJyNFgRaWDw,"Shopping, Accessories, Women's Clothing, Fashion",Nashville,"(10:0-17:30, 10:0-17:30, 10:0-17:0, None, 10:0...",0,36.126026,-86.845443,Jamie,37205,6,3.0,TN


<h4>**Analyse Business Data:Number of Restaurants per state

df_num_of_restaurants = df_business.select('state').groupBy('state').count().orderBy(f.desc("count"))

df_num_of_restaurants.show()

<h4>**Analyze top 3 restaurants in each state:

from pyspark.sql import Window

df_business_ranked = df_business.withColumn("rn",f.rank().over(Window.partitionBy("state").orderBy(f.desc("review_count"))))

pd = df_business_ranked.filter(f.col("rn") <= 3).toPandas()

pd.head(30)

<h4>**List the top restaurants in a state by the number of reviews:

In [18]:
df_state = df_business.filter(f.col("state") == "AZ")

In [19]:
df_business.filter(f.col("name") == "El Charro Cafe").agg(f.count("*")).show()

+--------+
|count(1)|
+--------+
|       6|
+--------+



<h4>**Numbers of restaurants in Arizona state per city

In [20]:
df_result=df_state.withColumn("rn",f.rank().over(Window.orderBy(f.desc("review_count"))))

df_arizona = df_business.filter(f.col("state") == "AZ")




df_result = df_business.filter(f.col("state") == "AZ").groupby("name").agg(f.count("review_count").alias("review")).orderBy(f.desc("review"))


pr = df_result.select("name","review").toPandas()

pr.head(10)

,name,review
0,Starbucks,63
1,Subway,50
2,Walgreens,44
3,McDonald's,41
4,eegee's,27
5,The UPS Store,26
6,Circle K,24
7,Burger King,23
8,Taco Bell,22
9,Supercuts,21


<h4>**Number of restaurants in Arizona state per city

In [21]:
df_az_cities = df_arizona.groupby("city").agg(f.count("*").alias("no_restaurants")).orderBy(f.desc("no_restaurants"))

pdz = df_az_cities.toPandas()
pdz.head(20)

,city,no_restaurants
0,Tucson,9249
1,Oro Valley,286
2,Marana,102
3,Sahuarita,69
4,Green Valley,69
5,Vail,51
6,Tuscon,18
7,Catalina,13
8,Mount Lemmon,7
9,TUCSON,6


<h4>**Select city with highest number of restaurants

In [22]:
df_az = df_business.filter(f.col("state") == "AZ").groupBy("name").agg(f.count("review_count").alias("count")).orderBy(f.desc("count"))

pdz = df_az.toPandas()
pdz.head(10)

df_ans = df_business.groupby("city").agg(f.count("*").alias("total_cities")).orderBy(f.desc("total_cities"))

df_ans.withColumn("rn",f.rank().over(Window.orderBy(f.desc("total_cities")))).filter(f.col("rn") == 1).drop("rn").select("*").show()

+------------+------------+
|        city|total_cities|
+------------+------------+
|Philadelphia|       14569|
+------------+------------+



<h4>**Broadcast Join

In [24]:
from pyspark.sql.functions import broadcast

df_best = df_business.join(broadcast(df_ans),"city",'inner')

df_best = df_best.groupBy("name","stars").agg(f.count("review_count").alias("review_count"))

df_best = df_best.filter(f.col("review_count") >= 10)
df_best = df_best.filter(f.col("stars") >= 3)

<h4>**Most Rated Italian restaurant in Philadelphia

In [26]:
df_business_Philadelphia = df_business.filter(df_business.city == 'Philadelphia')
df_business_italian = df_business_Philadelphia.filter(df_business.categories.contains('Italian'))

In [27]:
df_best_italian_restaurants = df_business_italian.groupBy("name").agg(f.count("review_count").alias("review_count"))

df_best_italian_restaurants = df_best_italian_restaurants.filter(df_best_italian_restaurants["review_count"] >= 5)

display(df_best_italian_restaurants)

DataFrame[name: string, review_count: bigint]